In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [27]:
map = gpd.read_file('../../Data/NYC_Geo_prayer.json')

In [28]:
map=map[['postalCode','PO_NAME','borough','geometry']]

In [4]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-1.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782492 entries, 0 to 13782491
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   vendor_id            object 
 1    pickup_datetime     object 
 2    dropoff_datetime    object 
 3    passenger_count     int64  
 4    trip_distance       float64
 5    pickup_longitude    float64
 6    pickup_latitude     float64
 7    rate_code           int64  
 8    store_and_fwd_flag  object 
 9    dropoff_longitude   float64
 10   dropoff_latitude    float64
 11   payment_type        object 
 12   fare_amount         float64
 13   surcharge           float64
 14   mta_tax             float64
 15   tip_amount          float64
 16   tolls_amount        float64
 17   total_amount        float64
dtypes: float64(11), int64(2), object(5)
memory usage: 1.8+ GB


In [6]:
df2=df.head(100)

In [10]:
df2.columns

Index(['vendor_id', ' pickup_datetime', ' dropoff_datetime',
       ' passenger_count', ' trip_distance', ' pickup_longitude',
       ' pickup_latitude', ' rate_code', ' store_and_fwd_flag',
       ' dropoff_longitude', ' dropoff_latitude', ' payment_type',
       ' fare_amount', ' surcharge', ' mta_tax', ' tip_amount',
       ' tolls_amount', ' total_amount'],
      dtype='object')

In [15]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)
df.head(3)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 817.3952524662018 seconds ---


In [16]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_geometry,dropoff_geometry
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.4,0.0,8.9,POINT (-73.99477 40.736828),POINT (-73.98222699999998 40.73179)
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.9,0.0,11.4,POINT (-73.982392 40.773382),POINT (-73.96044899999998 40.763995)
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.5,0.0,14.0,POINT (-73.98856999999998 40.739406),POINT (-73.986626 40.765217)


In [32]:
df2_start=df2[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [36]:
df2_end=df2[['dropoff_geometry']]

In [40]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 112.19443106651306 seconds ---


In [92]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 139.87490582466125 seconds ---


In [42]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1175.6654407978058 seconds ---


In [94]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1151.9103791713715 seconds ---


In [81]:
group_start = geo_processing_start.groupby(['postalCode']).describe()

In [98]:
group_end = geo_processing_end.value_counts(['postalCode'])

In [100]:
group_end.to_csv('../../Data/yellow_tripdata_2014-1-end-processed.csv')

In [85]:
group_start=group_start.columns = ['_'.join(col) for col in group2.columns.values]

In [87]:
group_start.to_csv('../../Data/yellow_tripdata_2014-1-start-processed.csv')